In [2]:
import pandas as pd
import numpy as np
from demoparser2 import DemoParser

In [3]:
demo1 = DemoParser("VS_KREATURES_Anubis.dem")
demo2 = DemoParser("VS_Eldsjalar_Vertigo.dem")

## Data prep

In [4]:
max_tick = demo1.parse_event("round_end")["tick"].max()

#### Test des différents parsers

In [5]:
event_df = demo1.parse_event("player_death", player=["X", "Y"], other=["total_rounds_played"])
all_events = demo1.parse_events(["all"])
ticks_df = demo1.parse_ticks(["X", "Y"])
stats = demo1.parse_grenades

In [6]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [7]:
ticks_df

,X,Y,tick,steamid,name
0,-400.0,2192.0,0,76561198111191178,-silentGG
1,-240.0,-1696.0,0,76561198264064168,JustAnotherL
2,-476.0,2216.0,0,76561198066661071,OzzieOzz
3,-608.0,2120.0,0,76561198259827471,godofbaldz
4,-192.0,-1608.0,0,76561199198706117,ADEODATUSS
...,...,...,...,...,...
2450495,NaN,NaN,245185,76561198376612283,BELDIYA00
2450496,NaN,NaN,245185,76561198014090099,fykseN
2450497,NaN,NaN,245185,76561198358546254,MasaIDK_
2450498,NaN,NaN,245185,76561198173360875,Spiritix


In [8]:
all_events

[('buytime_ended',
        tick
  0     2560
  1     6214
  2    11695
  3    19886
  4    24301
  5    29174
  6    35199
  7    43633
  8    50418
  9    60253
  10   66150
  11   70697
  12   79625
  13   88409
  14   93254
  15  101145
  16  110082
  17  115819
  18  125053
  19  133389
  20  141535
  21  149136
  22  159376
  23  170539
  24  181613
  25  188256
  26  196610
  27  204986
  28  214270
  29  222846
  30  231970),
 ('round_mvp',
      musickitid  musickitmvps  nomusic  reason    tick     user_name  \
  0           40             0        0       1    2498  JustAnotherL   
  1           70             0        0       3    8686   3PARABELLUM   
  2            0             0        0       1   14957      OzzieOzz   
  3            0             0        0       1   21292      Spiritix   
  4            0             0        0       1   26165    godofbaldz   
  5           40             0        0       1   32190  JustAnotherL   
  6            0             0       

#### Initialisation du DF Rounds et Ticks

In [9]:
rounds_ticks = demo1.parse_event('round_start')

In [10]:
rounds_ticks

,round,tick
0,1,0
1,1,2676
2,1,3120
3,1,3237
4,2,9134
5,3,15405
6,4,21740
7,5,26613
8,6,32638
9,7,41072


Isolation des ticks de début de rounds effectifs, sur faceit : 
- R1 = Knife round -> freezetime
- R2 = Warmup -> freezetime
- R3 = RR -> pas de freezetime
- R4 = 1er round

On supprime donc le tick 0 (knife) + le tick 2676 (warmup) + le tick  3120 (RR) pour ne démarrer qu'a partir du tick 3237 qui correspond au vrai premier round IG 


In [11]:
rounds_ticks = rounds_ticks.drop([0,1,2])

In [12]:
first_round_tick = rounds_ticks.loc[3, 'tick']
first_round_tick

3237

#### Freezetime end

In [13]:
freezetime_end_tick = demo1.parse_event("round_freeze_end")["tick"].drop([0,1]).tolist()

In [14]:
freezetime_end_tick

[4933,
 10414,
 18605,
 23020,
 27893,
 33918,
 42352,
 49137,
 58972,
 64869,
 69416,
 78344,
 87128,
 91973,
 99864,
 108801,
 114538,
 123772,
 132108,
 140254,
 147855,
 158095,
 169258,
 180332,
 186975,
 195329,
 203705,
 212989,
 221565,
 230689]

#### Map

In [15]:
map = pd.DataFrame([demo1.parse_header()])
map

,map_name,game_directory,demo_file_stamp,demo_version_name,network_protocol,allow_clientside_entities,demo_version_guid,addons,client_name,server_name,fullpackets_version,allow_clientside_particles
0,de_anubis,/home/hz00604/cs2/game/csgo,PBDEMS2 ,valve_demo_2,13994,true,8e9d71ab-04a1-4c01-bb61-acfede27c046,,SourceTV Demo,FACEIT.com register to play here,2,true


In [16]:
map2 = pd.DataFrame([demo2.parse_header()])
map2

,game_directory,server_name,client_name,addons,map_name,network_protocol,fullpackets_version,demo_version_guid,allow_clientside_particles,allow_clientside_entities,demo_file_stamp,demo_version_name
0,/home/hz01872/cs2/game/csgo,FACEIT.com register to play here,SourceTV Demo,,de_vertigo,13985,2,8e9d71ab-04a1-4c01-bb61-acfede27c046,true,true,PBDEMS2 ,valve_demo_2


#### Bomb plant 

In [17]:
bombsite = demo1.parse_event("bomb_planted", player=["last_place_name"])
bombsite

,site,tick,user_last_place_name,user_name,user_steamid
0,348,6821,BombsiteB,OzzieOzz,76561198066661071
1,348,13675,BombsiteB,OzzieOzz,76561198066661071
2,348,21598,BombsiteB,godofbaldz,76561198259827471
3,236,25321,BombsiteA,OzzieOzz,76561198066661071
4,236,38000,BombsiteA,OzzieOzz,76561198066661071
5,348,54620,BombsiteB,-silentGG,76561198111191178
6,348,61448,BombsiteB,godofbaldz,76561198259827471
7,236,68079,BombsiteA,-silentGG,76561198111191178
8,236,76374,BombsiteA,BELDIYA00,76561198376612283
9,236,84713,BombsiteA,-silentGG,76561198111191178


## Tables intermédiaires

#### Equipe et side

In [18]:
zobrux = ['OzzieOzz','-silentGG','godofbaldz','BELDIYA00','Spiritix']
Equipe = 'zobrux'

Une valeur de Team pour chaque joueur est ajoutée pour chaque changement de side, dans le cas de notre game témoin : 
- tick 3120 : Changement de side suite au Knife round (puis la game est RR donc vrai début de la partie au ticke 3237 mais pas de changement entre ces deux ticks donc pas de lignes supplémentaires)
- tick 85432 : Changement de side mi-temps du match
- tick 211293 : Changement de side Overtime

In [19]:
# Team_V2 = demo1.parse_event('player_team')
# Team_V2.sort_values(by=['tick','team'],inplace=True)
# Team_V2['team_name'] = np.where(Team_V2['user_name'].isin(zobrux),'Zobrux','Opposing_team')
# Team_V2['side'] = np.where(Team_V2['team'] == 2,'T','CT')
# Team_V2.drop(['disconnect','isbot','oldteam','silent','user_steamid'],axis=1,inplace=True)
# Team_V2.rename(columns={"user_name":"name"},inplace=True)

In [20]:
# Team_V2.head()

In [103]:
team_V3 = demo1.parse_ticks(["team_clan_name","team_name"])
team_V3.drop(team_V3[team_V3['tick'] < first_round_tick].index, inplace=True)
team_V3.drop(team_V3[team_V3['tick'] > max_tick].index, inplace=True)
team_V3.rename(columns={'team_name':'side'},inplace=True)

In [107]:
team_V3

,side,team_clan_name,tick,steamid,name
32310,TERRORIST,zobrux,3237,76561198111191178,-silentGG
32311,CT,KREATURES,3237,76561198264064168,JustAnotherL
32312,TERRORIST,zobrux,3237,76561198066661071,OzzieOzz
32313,TERRORIST,zobrux,3237,76561198259827471,godofbaldz
32314,CT,KREATURES,3237,76561199198706117,ADEODATUSS
...,...,...,...,...,...
2359395,TERRORIST,zobrux,236075,76561198376612283,BELDIYA00
2359396,CT,KREATURES,236075,76561198014090099,fykseN
2359397,CT,KREATURES,236075,76561198358546254,MasaIDK_
2359398,TERRORIST,zobrux,236075,76561198173360875,Spiritix


il semblerait que team 2 = Terrorist et team 3 = CT

#### Table Round winner

In [23]:
round_ends = demo1.parse_event("round_end", other=["total_rounds_played"])
round_ends.drop(round_ends[round_ends['tick'] <= first_round_tick].index, inplace=True)
round_ends.drop(columns={'round'}, inplace=True)

In [24]:
round_ends

,reason,tick,total_rounds_played,winner
1,bomb_defused,8686,1,CT
2,ct_killed,14957,2,T
3,ct_killed,21292,3,T
4,ct_killed,26165,4,T
5,t_killed,32190,5,CT
6,bomb_exploded,40624,6,T
7,t_killed,47409,7,CT
8,bomb_exploded,57244,8,T
9,ct_killed,63141,9,T
10,ct_killed,67688,10,T


#### Table kills

In [25]:
kills_df = demo1.parse_event("player_death", player=["X", "Y"], other=["total_rounds_played"])

In [26]:
kills_df.columns

Index(['assistedflash', 'assister_X', 'assister_Y', 'assister_name',
       'assister_steamid', 'attacker_X', 'attacker_Y', 'attacker_name',
       'attacker_steamid', 'attackerblind', 'distance', 'dmg_armor',
       'dmg_health', 'dominated', 'headshot', 'hitgroup', 'noreplay',
       'noscope', 'penetrated', 'revenge', 'thrusmoke', 'tick',
       'total_rounds_played', 'user_X', 'user_Y', 'user_name', 'user_steamid',
       'weapon', 'weapon_fauxitemid', 'weapon_itemid',
       'weapon_originalowner_xuid', 'wipe'],
      dtype='object')

In [27]:
kills_df

,assistedflash,assister_X,assister_Y,assister_name,assister_steamid,attacker_X,attacker_Y,attacker_name,attacker_steamid,attackerblind,...,total_rounds_played,user_X,user_Y,user_name,user_steamid,weapon,weapon_fauxitemid,weapon_itemid,weapon_originalowner_xuid,wipe
0,False,-167.055969,237.575424,Spiritix,76561198173360875,-199.970703,259.122498,OzzieOzz,76561198066661071,False,...,0,-199.327637,210.264328,fykseN,76561198014090099,knife_butterfly,17293822569105523203,35477304051,,0
1,False,-122.406067,180.313324,JustAnotherL,76561198264064168,-90.374817,180.395264,ADEODATUSS,76561199198706117,False,...,0,-100.432251,212.426514,-silentGG,76561198111191178,knife_t,17293822569102704699,0,,0
2,False,-52.514343,267.781067,BELDIYA00,76561198376612283,-50.618103,138.054077,godofbaldz,76561198259827471,False,...,0,-37.088135,106.022820,3PARABELLUM,76561199074956403,knife_skeleton,17293822569103491597,35167355779,,0
3,False,-37.319031,106.022820,3PARABELLUM,76561199074956403,-60.121765,136.473846,ADEODATUSS,76561199198706117,False,...,0,-72.574768,172.412903,BELDIYA00,76561198376612283,knife_t,17293822569102704699,0,,0
4,False,-199.968750,154.260818,MasaIDK_,76561198358546254,-153.121460,136.579834,JustAnotherL,76561198264064168,False,...,0,-101.537781,164.830856,Spiritix,76561198173360875,knife_stiletto,17293822569106571786,34873144654,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,False,NaN,NaN,None,None,-1843.828613,7.268648,godofbaldz,76561198259827471,False,...,29,-1211.707642,998.469543,MasaIDK_,76561198358546254,awp,17293822569116598281,33914766179,,0
205,False,NaN,NaN,None,None,-1903.309448,284.353912,godofbaldz,76561198259827471,False,...,29,-639.289856,701.306396,ADEODATUSS,76561199198706117,awp,17293822569116598281,33914766179,,0
206,False,NaN,NaN,None,None,-1827.925171,112.288429,godofbaldz,76561198259827471,False,...,29,-1188.051514,985.835938,JustAnotherL,76561198264064168,awp,17293822569116598281,33914766179,,0
207,False,NaN,NaN,None,None,-891.219482,320.774597,OzzieOzz,76561198066661071,False,...,29,-971.060059,1109.107056,fykseN,76561198014090099,ak47,17293822569133768711,35297643185,,0


#### Table Eco

In [28]:
eco = demo1.parse_ticks(["current_equip_value", "total_rounds_played"], ticks=freezetime_end_tick)

In [29]:
eco

,total_rounds_played,current_equip_value,tick,steamid,name
0,0,850,4933,76561198111191178,-silentGG
1,0,850,4933,76561198264064168,JustAnotherL
2,0,1150,4933,76561198066661071,OzzieOzz
3,0,850,4933,76561198259827471,godofbaldz
4,0,850,4933,76561199198706117,ADEODATUSS
...,...,...,...,...,...
295,29,5100,230689,76561198376612283,BELDIYA00
296,29,3650,230689,76561198014090099,fykseN
297,29,5500,230689,76561198358546254,MasaIDK_
298,29,5100,230689,76561198173360875,Spiritix


La fonction "total_rounds_played" démarre de 0 a 29 on va y ajouter 1 pour que ça corresponde aux rounds de notre partie

In [30]:
eco['total_rounds_played'] = eco['total_rounds_played'] + 1

la fonction "current_equip_value" compte la valeur du glock (T) et de l'usp (CT) dans la valeur totale il faut donc la supprimer

In [31]:
eco['current_equip_value'] = eco['current_equip_value'] - 200

In [32]:
eco

,total_rounds_played,current_equip_value,tick,steamid,name
0,1,650,4933,76561198111191178,-silentGG
1,1,650,4933,76561198264064168,JustAnotherL
2,1,950,4933,76561198066661071,OzzieOzz
3,1,650,4933,76561198259827471,godofbaldz
4,1,650,4933,76561199198706117,ADEODATUSS
...,...,...,...,...,...
295,30,4900,230689,76561198376612283,BELDIYA00
296,30,3450,230689,76561198014090099,fykseN
297,30,5300,230689,76561198358546254,MasaIDK_
298,30,4900,230689,76561198173360875,Spiritix


On va enrichir avec les notions d'équipe et de side 

In [33]:
eco_by_players = pd.merge(eco,team_V3, on=['tick','name','steamid'])

In [34]:
eco_by_players

,total_rounds_played,current_equip_value,tick,steamid,name,side,team_clan_name
0,1,650,4933,76561198111191178,-silentGG,TERRORIST,zobrux
1,1,650,4933,76561198264064168,JustAnotherL,CT,KREATURES
2,1,950,4933,76561198066661071,OzzieOzz,TERRORIST,zobrux
3,1,650,4933,76561198259827471,godofbaldz,TERRORIST,zobrux
4,1,650,4933,76561199198706117,ADEODATUSS,CT,KREATURES
...,...,...,...,...,...,...,...
295,30,4900,230689,76561198376612283,BELDIYA00,TERRORIST,zobrux
296,30,3450,230689,76561198014090099,fykseN,CT,KREATURES
297,30,5300,230689,76561198358546254,MasaIDK_,CT,KREATURES
298,30,4900,230689,76561198173360875,Spiritix,TERRORIST,zobrux


On va également enrichir a l'aide des notions de round winner et win_reason

In [35]:
eco_by_players = pd.merge_asof(eco_by_players,round_ends[['tick','winner','reason']], on='tick', direction='forward')

In [36]:
eco_by_players

,total_rounds_played,current_equip_value,tick,steamid,name,side,team_clan_name,winner,reason
0,1,650,4933,76561198111191178,-silentGG,TERRORIST,zobrux,CT,bomb_defused
1,1,650,4933,76561198264064168,JustAnotherL,CT,KREATURES,CT,bomb_defused
2,1,950,4933,76561198066661071,OzzieOzz,TERRORIST,zobrux,CT,bomb_defused
3,1,650,4933,76561198259827471,godofbaldz,TERRORIST,zobrux,CT,bomb_defused
4,1,650,4933,76561199198706117,ADEODATUSS,CT,KREATURES,CT,bomb_defused
...,...,...,...,...,...,...,...,...,...
295,30,4900,230689,76561198376612283,BELDIYA00,TERRORIST,zobrux,T,ct_killed
296,30,3450,230689,76561198014090099,fykseN,CT,KREATURES,T,ct_killed
297,30,5300,230689,76561198358546254,MasaIDK_,CT,KREATURES,T,ct_killed
298,30,4900,230689,76561198173360875,Spiritix,TERRORIST,zobrux,T,ct_killed


On va grouper par team pour avoir l'infos également

In [37]:
eco_by_team = eco_by_players.groupby(['total_rounds_played','team_clan_name','tick','side','winner','reason'])['current_equip_value'].sum().reset_index()

In [38]:
eco_by_team.head()

,total_rounds_played,team_clan_name,tick,side,winner,reason,current_equip_value
0,1,KREATURES,4933,CT,CT,bomb_defused,3250
1,1,zobrux,4933,TERRORIST,CT,bomb_defused,3600
2,2,KREATURES,10414,CT,T,ct_killed,19850
3,2,zobrux,10414,TERRORIST,T,ct_killed,15000
4,3,KREATURES,18605,CT,T,ct_killed,7500


## Calculs des KPI

#### Analyse par joueurs

##### Stats kills

In [39]:
kills_df = demo1.parse_event("player_death", other=["total_rounds_played"])
kills_df = kills_df[kills_df['tick'] >= first_round_tick]
kills_df['total_rounds_played'] = kills_df['total_rounds_played'] + 1

In [40]:
# on enrichie avec l'équipe et le side du tueur
kills_df = pd.merge(kills_df,team_V3, left_on=['tick','attacker_name'],right_on=['tick','name'])

In [41]:
kills_df = kills_df.rename(columns={'team_clan_name':'attacker_team_name','side':'attacker_side'})
kills_df = kills_df.drop(columns=['name','steamid'])

In [42]:
# on enrichie avec l'équipe et le side du tué
kills_df = pd.merge(kills_df,team_V3, left_on=['tick','user_name'],right_on=['tick','name'])

In [43]:
kills_df = kills_df.rename(columns={'team_clan_name':'user_team_name','side':'user_side'})
kills_df = kills_df.drop(columns=['name','steamid'])

In [44]:
df_global = kills_df.merge(eco_by_players, left_on=['total_rounds_played','attacker_name'], right_on=['total_rounds_played','name'])

In [45]:
df_global['map'] = demo1.parse_header()['map_name']

In [46]:
df_global.head()

,assistedflash,assister_name,assister_steamid,attacker_name,attacker_steamid,attackerblind,distance,dmg_armor,dmg_health,dominated,...,user_team_name,current_equip_value,tick_y,steamid,name,side,team_clan_name,winner,reason,map
0,False,None,None,JustAnotherL,76561198264064168,False,16.071526,0,124,0,...,zobrux,650,4933,76561198264064168,JustAnotherL,CT,KREATURES,CT,bomb_defused,de_anubis
1,False,JustAnotherL,76561198264064168,MasaIDK_,76561198358546254,False,10.993015,0,129,0,...,zobrux,650,4933,76561198358546254,MasaIDK_,CT,KREATURES,CT,bomb_defused,de_anubis
2,False,None,None,OzzieOzz,76561198066661071,False,17.510454,0,96,0,...,KREATURES,950,4933,76561198066661071,OzzieOzz,TERRORIST,zobrux,CT,bomb_defused,de_anubis
3,False,None,None,BELDIYA00,76561198376612283,False,14.075491,0,101,0,...,KREATURES,650,4933,76561198376612283,BELDIYA00,TERRORIST,zobrux,CT,bomb_defused,de_anubis
4,False,None,None,godofbaldz,76561198259827471,False,5.924605,9,17,0,...,KREATURES,650,4933,76561198259827471,godofbaldz,TERRORIST,zobrux,CT,bomb_defused,de_anubis


In [47]:
df_global.columns

Index(['assistedflash', 'assister_name', 'assister_steamid', 'attacker_name',
       'attacker_steamid', 'attackerblind', 'distance', 'dmg_armor',
       'dmg_health', 'dominated', 'headshot', 'hitgroup', 'noreplay',
       'noscope', 'penetrated', 'revenge', 'thrusmoke', 'tick_x',
       'total_rounds_played', 'user_name', 'user_steamid', 'weapon',
       'weapon_fauxitemid', 'weapon_itemid', 'weapon_originalowner_xuid',
       'wipe', 'attacker_side', 'attacker_team_name', 'user_side',
       'user_team_name', 'current_equip_value', 'tick_y', 'steamid', 'name',
       'side', 'team_clan_name', 'winner', 'reason', 'map'],
      dtype='object')

#### Extraction des infos spécifique depuis le demo1 event

##### Trade

In [136]:
trade = demo1.parse_event("player_death", other=["game_time", "round_start_time",'total_rounds_played'])
trade.drop(trade[trade['tick'] < first_round_tick].index, inplace=True)
trade["player_died_time"] = trade["game_time"] - trade["round_start_time"]
trade
trade  = pd.merge(trade,team_V3[['team_clan_name','side','name','tick']], left_on=["attacker_name",'tick'], right_on=['name','tick'])
trade
trade = trade.rename(columns={'team_clan_name':'team_clan_name_attacker','side':'side_attacker'})
trade  = pd.merge(trade,team_V3[['team_clan_name','side','name','tick']], left_on=["user_name",'tick'], right_on=['name','tick'])
trade = trade.rename(columns={'team_clan_name':'team_clan_name_user','side':'side_user'})
trade = trade.loc[:, ["total_rounds_played","tick","attacker_name","user_name",'team_clan_name_user',"side_user","player_died_time"]]
trade.head(30)

,total_rounds_played,tick,attacker_name,user_name,team_clan_name_user,side_user,player_died_time
0,0,6171,JustAnotherL,-silentGG,zobrux,TERRORIST,19.328125
1,0,6252,MasaIDK_,Spiritix,zobrux,TERRORIST,20.593750
2,0,6407,OzzieOzz,fykseN,KREATURES,CT,23.015625
3,0,6484,BELDIYA00,JustAnotherL,KREATURES,CT,24.218750
4,0,6622,godofbaldz,MasaIDK_,KREATURES,CT,26.375000
5,0,7119,3PARABELLUM,OzzieOzz,zobrux,TERRORIST,34.140625
6,0,7505,ADEODATUSS,godofbaldz,zobrux,TERRORIST,40.171875
7,0,7808,BELDIYA00,ADEODATUSS,KREATURES,CT,44.906250
8,0,7841,3PARABELLUM,BELDIYA00,zobrux,TERRORIST,45.421875
9,1,12878,Spiritix,ADEODATUSS,KREATURES,CT,38.484375


In [241]:
# Initialiser des dictionnaires pour stocker les traded deaths et trade kills par joueur
traded_deaths = {}
trade_kills = {}

# Parcourir toutes les lignes du DataFrame pour chercher les morts des terroristes
for i, row in trade.iterrows():
    if row['side_user'] == 'TERRORIST':
        # Extraire les informations de la ligne courante
        terrorist_name = row['user_name']
        attacker_name = row['attacker_name']
        death_time = row['player_died_time']
        round_number = row['total_rounds_played']
        
        # Chercher si l'attaquant (CT) est tué dans les 2 secondes après avoir tué le terroriste
        subsequent_kills = trade[(trade['user_name'] == attacker_name) & 
                                 (trade['player_died_time'] > death_time) & 
                                 (trade['player_died_time'] <= death_time + 2)]
        
        # Si un traded death est trouvé
        if not subsequent_kills.empty:
            # Assurer que la mort de l'attaquant se produit dans le même round que la mort du terroriste
            if all(subsequent_kills['total_rounds_played'] == round_number):
                if terrorist_name not in traded_deaths:
                    traded_deaths[terrorist_name] = {'count': 0, 'rounds': []}
                traded_deaths[terrorist_name]['count'] += 1
                traded_deaths[terrorist_name]['rounds'].append(round_number)

                # Chercher si le trade kill vient d'un coéquipier du terroriste
                for _, sub_kill in subsequent_kills.iterrows():
                    trade_killer = sub_kill['attacker_name']
                    if trade_killer != terrorist_name:
                        if trade_killer not in trade_kills:
                            trade_kills[trade_killer] = {'count': 0, 'rounds': []}
                        trade_kills[trade_killer]['count'] += 1
                        trade_kills[trade_killer]['rounds'].append(round_number)

# Convertir les dictionnaires en DataFrames pour afficher les résultats
traded_deaths_list = [(player, data['count']) for player, data in traded_deaths.items()]
trade_kills_list = [(player, data['count']) for player, data in trade_kills.items()]

traded_deaths_df = pd.DataFrame(traded_deaths_list, columns=['name', 'Traded Deaths'])
trade_kills_df = pd.DataFrame(trade_kills_list, columns=['name', 'Trade Kills'])

# Afficher les résultats
traded_deaths_df


,name,Traded Deaths
0,godofbaldz,2
1,-silentGG,3
2,OzzieOzz,1
3,JustAnotherL,4
4,fykseN,2
5,3PARABELLUM,2
6,MasaIDK_,1
7,ADEODATUSS,1


In [242]:
trade_kills_df 

,name,Trade Kills
0,OzzieOzz,2
1,godofbaldz,1
2,-silentGG,1
3,fykseN,1
4,ADEODATUSS,5
5,JustAnotherL,2
6,MasaIDK_,1
7,3PARABELLUM,1
8,BELDIYA00,2


##### Utils efficency

In [155]:
utils_stats = ["total_rounds_played","utility_damage_total","enemies_flashed_total"]

stats_utils = demo1.parse_ticks(utils_stats, ticks=[max_tick])
stats_utils = stats_utils.rename(columns={"utility_damage_total":"Utility Damages","enemies_flashed_total":"Flashed Ennemies"})
stats_utils = stats_utils.drop(columns={"tick"})
nouvel_ordre_colonnes = [
    'name', 'steamid', "Utility Damages","Flashed Ennemies"
]
stats_utils = stats_utils[nouvel_ordre_colonnes]
stats_utils

,name,steamid,Utility Damages,Flashed Ennemies
0,-silentGG,76561198111191178,175,7
1,JustAnotherL,76561198264064168,106,13
2,OzzieOzz,76561198066661071,75,5
3,godofbaldz,76561198259827471,79,14
4,ADEODATUSS,76561199198706117,34,9
5,BELDIYA00,76561198376612283,257,6
6,fykseN,76561198014090099,163,8
7,MasaIDK_,76561198358546254,85,3
8,Spiritix,76561198173360875,230,2
9,3PARABELLUM,76561199074956403,166,11


Flash assist

In [177]:
flash_assist_count = df_global[df_global['assistedflash'] == True].groupby('assister_name').size().reset_index(name='Flash_assist')
flash_assist_count = flash_assist_count.rename(columns={'assister_name':'name'})
flash_assist_count

,name,Flash_assist
0,ADEODATUSS,1
1,JustAnotherL,1
2,godofbaldz,1


##### Opening duels

In [209]:
# Identifier le premier kill de chaque round
first_duel = trade.groupby('total_rounds_played').first().reset_index()

In [210]:
first_duel

,total_rounds_played,tick,attacker_name,user_name,team_clan_name_user,side_user,player_died_time
0,0,6171,JustAnotherL,-silentGG,zobrux,TERRORIST,19.328125
1,1,12878,Spiritix,ADEODATUSS,KREATURES,CT,38.484375
2,2,19404,Spiritix,3PARABELLUM,KREATURES,CT,12.468750
3,3,24552,-silentGG,fykseN,KREATURES,CT,23.921875
4,4,28801,MasaIDK_,-silentGG,zobrux,TERRORIST,14.171875
5,5,34832,godofbaldz,JustAnotherL,KREATURES,CT,14.265625
6,6,43818,JustAnotherL,godofbaldz,zobrux,TERRORIST,22.890625
7,7,51119,godofbaldz,ADEODATUSS,KREATURES,CT,30.953125
8,8,60808,OzzieOzz,JustAnotherL,KREATURES,CT,28.671875
9,9,66126,BELDIYA00,JustAnotherL,KREATURES,CT,19.625000


In [233]:
# Initialiser un DataFrame pour stocker les résultats par joueur
opening_stats = pd.DataFrame({
    'name': trade['attacker_name'].unique(),
    'entry_attempts(T)': 0,
    'entry_successes(T)': 0,
    'open_attempts(CT)': 0,
    'open_successes(CT)': 0
})

# Parcourir chaque premier kill pour déterminer si c'est une tentative d'entry (T) ou d'open (CT)
for _, row in first_duel.iterrows():
    attacker = row['attacker_name']
    user = row['user_name']
    side_user = row['side_user']

    # Si le joueur tué est un CT, c'est une tentative d'entry pour un Terrorist
    if side_user == 'CT':
        # C'est une tentative d'entry pour l'attaquant
        opening_stats.loc[opening_stats['name'] == attacker, 'entry_attempts(T)'] += 1
        opening_stats.loc[opening_stats['name'] == attacker, 'entry_successes(T)'] += 1
        opening_stats.loc[opening_stats['name'] == user, 'open_attempts(CT)'] += 1


    # Si le joueur tué est un Terroriste, c'est une tentative d'open pour un CT
    elif side_user == 'TERRORIST':
        # C'est une tentative d'open pour l'attaquant
        opening_stats.loc[opening_stats['name'] == attacker, 'open_attempts(CT)'] += 1
        opening_stats.loc[opening_stats['name'] == attacker, 'open_successes(CT)'] += 1
        opening_stats.loc[opening_stats['name'] == user, 'entry_attempts(T)'] += 1

        

# Afficher le résultat
opening_stats


,name,entry_attempts(T),entry_successes(T),open_attempts(CT),open_successes(CT)
0,JustAnotherL,3,1,6,2
1,MasaIDK_,1,1,2,1
2,OzzieOzz,2,1,4,0
3,BELDIYA00,1,1,2,2
4,godofbaldz,6,4,5,3
5,3PARABELLUM,7,3,2,1
6,ADEODATUSS,1,1,2,0
7,Spiritix,3,2,3,1
8,-silentGG,3,1,1,1
9,fykseN,3,2,3,2


In [234]:
opening_stats["%_entry_success(T)"] = round((opening_stats["entry_successes(T)"] / opening_stats["entry_attempts(T)"])*100,0)
opening_stats["%_open_success(CT)"] = round((opening_stats["open_successes(CT)"] / opening_stats["open_attempts(CT)"])*100,0)

In [236]:
column_order=['name','entry_attempts(T)','entry_successes(T)','%_entry_success(T)','open_attempts(CT)','open_successes(CT)','%_open_success(CT)']
opening_stats = opening_stats[column_order]

In [237]:
opening_stats

,name,entry_attempts(T),entry_successes(T),%_entry_success(T),open_attempts(CT),open_successes(CT),%_open_success(CT)
0,JustAnotherL,3,1,33.0,6,2,33.0
1,MasaIDK_,1,1,100.0,2,1,50.0
2,OzzieOzz,2,1,50.0,4,0,0.0
3,BELDIYA00,1,1,100.0,2,2,100.0
4,godofbaldz,6,4,67.0,5,3,60.0
5,3PARABELLUM,7,3,43.0,2,1,50.0
6,ADEODATUSS,1,1,100.0,2,0,0.0
7,Spiritix,3,2,67.0,3,1,33.0
8,-silentGG,3,1,33.0,1,1,100.0
9,fykseN,3,2,67.0,3,2,67.0


##### Clutchs

In [54]:
df_global.columns

Index(['assistedflash', 'assister_name', 'assister_steamid', 'attacker_name',
       'attacker_steamid', 'attackerblind', 'distance', 'dmg_armor',
       'dmg_health', 'dominated', 'headshot', 'hitgroup', 'noreplay',
       'noscope', 'penetrated', 'revenge', 'thrusmoke', 'tick_x',
       'total_rounds_played', 'user_name', 'user_steamid', 'weapon',
       'weapon_fauxitemid', 'weapon_itemid', 'weapon_originalowner_xuid',
       'wipe', 'attacker_side', 'attacker_team_name', 'user_side',
       'user_team_name', 'current_equip_value', 'tick_y', 'steamid', 'name',
       'side', 'team_clan_name', 'winner', 'reason', 'map'],
      dtype='object')

In [55]:
df_global[['attacker_name','user_name','tick_x','total_rounds_played']].head(20)

,attacker_name,user_name,tick_x,total_rounds_played
0,JustAnotherL,-silentGG,6171,1
1,MasaIDK_,Spiritix,6252,1
2,OzzieOzz,fykseN,6407,1
3,BELDIYA00,JustAnotherL,6484,1
4,godofbaldz,MasaIDK_,6622,1
5,3PARABELLUM,OzzieOzz,7119,1
6,ADEODATUSS,godofbaldz,7505,1
7,BELDIYA00,ADEODATUSS,7808,1
8,3PARABELLUM,BELDIYA00,7841,1
9,Spiritix,ADEODATUSS,12878,2


In [56]:
# Initialiser un DataFrame pour suivre les clutches
clutch_stats = pd.DataFrame(columns=["player_name", "ct_clutches_attempted", "ct_clutches_won", "t_clutches_attempted", "t_clutches_won"])
clutch_stats.set_index("player_name", inplace=True)

# Fonction pour mettre à jour les statistiques de clutch pour CT ou T
def update_clutch_stats(player_name, team, won):
    if player_name not in clutch_stats.index:
        clutch_stats.loc[player_name] = [0, 0, 0, 0]  # Initialiser avec 0 tentatives et 0 réussites
    
    if team == "CT":
        # Incrémenter les clutchs tentés pour CT
        clutch_stats.loc[player_name, "ct_clutches_attempted"] += 1
        if won:
            clutch_stats.loc[player_name, "ct_clutches_won"] += 1
    elif team == "TERRORIST":
        # Incrémenter les clutchs tentés pour T
        clutch_stats.loc[player_name, "t_clutches_attempted"] += 1
        if won:
            clutch_stats.loc[player_name, "t_clutches_won"] += 1

X = 1

def find_if_1vx(deaths, round_idx, round_ends, df, X):
    # Filtrer les décès correspondant au round actuel
    round_deaths = deaths[deaths["total_rounds_played"] == round_idx]
    
    if round_deaths.empty:
        return None
    
    # Filtrer les ticks du round
    subdf = df[df["tick"].isin(round_deaths["tick"].to_list())]

    # Vérifier que nous avons un vainqueur dans round_ends pour ce round
    round_winner_info = round_ends.loc[round_ends['total_rounds_played'] == round_idx]
    if round_winner_info.empty:
        print(f"No winner info for round {round_idx}, skipping...")
        return None

    # Débogage pour vérifier les informations de vainqueur
    round_winner = round_winner_info["winner"].values[0]
    print(f"Round {round_idx}: Winner is {round_winner}")

    # Suivre les joueurs vivants à chaque étape du round
    for tick in sorted(subdf["tick"].unique()):
        current_tick_df = subdf[subdf["tick"] == tick]
        ct_alive = current_tick_df[(current_tick_df["team_name"] == "CT") & (current_tick_df["is_alive"] == True)]
        t_alive = current_tick_df[(current_tick_df["team_name"] == "TERRORIST") & (current_tick_df["is_alive"] == True)]
        
        # Vérification de la condition 1vX
        if len(ct_alive) == 1 and len(t_alive) == X:
            print(f"Detected 1v{X} situation for CT in round {round_idx}")
            if round_winner == "CT":  # CT a gagné
                print(f"CT won the round in a 1v{X}")
                update_clutch_stats(ct_alive["name"].iloc[0], "CT", won=True)  # CT wins the clutch
                update_clutch_stats(t_alive["name"].iloc[0], "TERRORIST", won=False)  # T loses the clutch
            else:
                print(f"T won the round in a 1v{X}")
                update_clutch_stats(ct_alive["name"].iloc[0], "CT", won=False)  # CT loses the clutch
                update_clutch_stats(t_alive["name"].iloc[0], "TERRORIST", won=True)  # T wins the clutch
        elif len(t_alive) == 1 and len(ct_alive) == X:
            print(f"Detected 1v{X} situation for TERRORIST in round {round_idx}")
            if round_winner == "T":  # T a gagné
                print(f"T won the round in a 1v{X}")
                update_clutch_stats(t_alive["name"].iloc[0], "TERRORIST", won=True)  # T wins the clutch
                update_clutch_stats(ct_alive["name"].iloc[0], "CT", won=False)  # CT loses the clutch
            else:
                print(f"CT won the round in a 1v{X}")
                update_clutch_stats(t_alive["name"].iloc[0], "TERRORIST", won=False)  # T loses the clutch
                update_clutch_stats(ct_alive["name"].iloc[0], "CT", won=True)  # CT wins the clutch

# Chargement des événements avec la nouvelle colonne 'total_rounds_played'
deaths = demo1.parse_event("player_death", other=["total_rounds_played"])
deaths['total_rounds_played'] = deaths['total_rounds_played'] + 1
deaths.drop(deaths[deaths['tick'] <= first_round_tick].index, inplace=True)

round_ends = demo1.parse_event("round_end", other=["total_rounds_played"])
round_ends.drop(round_ends[round_ends['tick'] <= first_round_tick].index, inplace=True)

# Réinitialiser les indices de 'round_ends'
round_ends = round_ends.reset_index(drop=True)

df = demo1.parse_ticks(["is_alive", "team_name", "team_rounds_total"], ticks=deaths["tick"].to_list())
max_round = deaths["total_rounds_played"].max() + 1

# Parcourir tous les rounds
for round_idx in range(1, max_round):
    print(f"Checking for 1v{X} in round {round_idx}")
    clutcher_steamid = find_if_1vx(deaths, round_idx, round_ends, df, X)
    if clutcher_steamid is not None:
        print(f"Round: {round_idx} {clutcher_steamid} clutched a 1v{X}")

# Afficher le DataFrame final des statistiques de clutch
print(clutch_stats)

Checking for 1v1 in round 1
Round 1: Winner is CT
Detected 1v1 situation for CT in round 1
CT won the round in a 1v1
Checking for 1v1 in round 2
Round 2: Winner is T
Checking for 1v1 in round 3
Round 3: Winner is T
Checking for 1v1 in round 4
Round 4: Winner is T
Checking for 1v1 in round 5
Round 5: Winner is CT
Checking for 1v1 in round 6
Round 6: Winner is T
Checking for 1v1 in round 7
Round 7: Winner is CT
Detected 1v1 situation for CT in round 7
CT won the round in a 1v1
Checking for 1v1 in round 8
Round 8: Winner is T
Checking for 1v1 in round 9
Round 9: Winner is T
Checking for 1v1 in round 10
Round 10: Winner is T
Checking for 1v1 in round 11
Round 11: Winner is T
Detected 1v1 situation for CT in round 11
T won the round in a 1v1
Checking for 1v1 in round 12
Round 12: Winner is T
Checking for 1v1 in round 13
Round 13: Winner is T
Checking for 1v1 in round 14
Round 14: Winner is T
Checking for 1v1 in round 15
Round 15: Winner is T
Checking for 1v1 in round 16
Round 16: Winner is 

In [57]:
clutch_stats

,ct_clutches_attempted,ct_clutches_won,t_clutches_attempted,t_clutches_won
player_name,,,,
3PARABELLUM,2,1,0,0
BELDIYA00,0,0,3,2
ADEODATUSS,2,1,0,0
Spiritix,0,0,1,0
JustAnotherL,1,0,0,0
OzzieOzz,0,0,1,1


#### Score d'eco

In [58]:
df_global.columns

Index(['assistedflash', 'assister_name', 'assister_steamid', 'attacker_name',
       'attacker_steamid', 'attackerblind', 'distance', 'dmg_armor',
       'dmg_health', 'dominated', 'headshot', 'hitgroup', 'noreplay',
       'noscope', 'penetrated', 'revenge', 'thrusmoke', 'tick_x',
       'total_rounds_played', 'user_name', 'user_steamid', 'weapon',
       'weapon_fauxitemid', 'weapon_itemid', 'weapon_originalowner_xuid',
       'wipe', 'attacker_side', 'attacker_team_name', 'user_side',
       'user_team_name', 'current_equip_value', 'tick_y', 'steamid', 'name',
       'side', 'team_clan_name', 'winner', 'reason', 'map'],
      dtype='object')

In [59]:
#ANTI X KILLS EN FONCTION DE LA MONNAIE DE L'ADVERSAIRE'

# Fusionner les DataFrames sur les colonnes 'user_steamid' et 'steamid' et 'total_rounds_played'
df_merged = pd.merge(kills_df, eco_by_players, how='left', left_on=['attacker_name', 'total_rounds_played'], right_on=['name', 'total_rounds_played'])

# Filtrer les rounds 1 et 13 (pistol rounds) en les incluant dans une nouvelle catégorie
def categorize_kill(row):
    if row['total_rounds_played'] in [1, 13]:
        return 'pistol round'
    elif row['current_equip_value'] < 1000:
        return 'eco'
    elif row['current_equip_value'] < 3500:
        return 'force buy'
    else:
        return 'full buy'

# Appliquer la fonction de catégorisation
df_merged['kill_category'] = df_merged.apply(categorize_kill, axis=1)

# Utilisation de pivot_table pour compter les kills par catégorie et par joueur
pivot_df = df_merged.pivot_table(index='attacker_name', columns='kill_category', aggfunc='size', fill_value=0)

# Afficher le résultat
pivot_df

kill_category,eco,force buy,full buy,pistol round
attacker_name,,,,
-silentGG,1,2,14,0
3PARABELLUM,0,3,11,5
ADEODATUSS,0,1,21,2
BELDIYA00,0,1,24,2
JustAnotherL,1,0,13,1
MasaIDK_,1,5,12,1
OzzieOzz,1,3,14,1
Spiritix,0,1,17,0
fykseN,2,2,12,1


In [60]:
#X KILLS EN FONCTION DE LA MONNAIE DE L'ADVERSAIRE'

# Fusionner les DataFrames sur les colonnes 'user_steamid' et 'steamid' et 'total_rounds_played'
df_merged = pd.merge(kills_df, eco_by_players, how='left', left_on=['user_name', 'total_rounds_played'], right_on=['name', 'total_rounds_played'])

# Filtrer les rounds 1 et 13 (pistol rounds) en les incluant dans une nouvelle catégorie
def categorize_kill(row):
    if row['total_rounds_played'] in [1, 13]:
        return 'pistol round'
    elif row['current_equip_value'] < 1000:
        return 'eco'
    elif row['current_equip_value'] < 3500:
        return 'force buy'
    else:
        return 'full buy'

# Appliquer la fonction de catégorisation
df_merged['kill_category'] = df_merged.apply(categorize_kill, axis=1)

# Utilisation de pivot_table pour compter les kills par catégorie et par joueur
pivot_df = df_merged.pivot_table(index='attacker_name', columns='kill_category', aggfunc='size', fill_value=0)

# Afficher le résultat
pivot_df

kill_category,eco,force buy,full buy,pistol round
attacker_name,,,,
-silentGG,1,3,13,0
3PARABELLUM,2,3,9,5
ADEODATUSS,3,0,19,2
BELDIYA00,4,2,19,2
JustAnotherL,0,2,12,1
MasaIDK_,5,2,11,1
OzzieOzz,3,3,12,1
Spiritix,2,8,8,0
fykseN,1,1,14,1


In [61]:
# Fusionner les données de kills avec l'économie totale de l'équipe de l'attaquant
df_merged_team = pd.merge(kills_df, eco_by_team, how='left', left_on=['total_rounds_played', 'user_team_name'], right_on=['total_rounds_played', 'team_clan_name'])

# Filtrer les rounds 1 et 13 (pistol rounds) en les incluant dans une nouvelle catégorie
def categorize_kill(row):
    if row['total_rounds_played'] in [1, 13]:
        return 'Pistol round'
    elif row['current_equip_value'] <= 3500:
        return 'Anti eco kill'
    elif row['current_equip_value'] <= 18000:
        return 'Anti force buy kill'
    else:
        return 'Full buy kill'

# Appliquer la fonction de catégorisation
df_merged_team['kill_category'] = df_merged_team.apply(categorize_kill, axis=1)

# Utilisation de pivot_table pour compter les kills par catégorie et par joueur
pivot_df_team = df_merged_team.pivot_table(index='attacker_name', columns='kill_category', aggfunc='size', fill_value=0)

# Afficher le résultat
pivot_df_team

kill_category,Anti eco kill,Anti force buy kill,Full buy kill,Pistol round
attacker_name,,,,
-silentGG,1,2,14,0
3PARABELLUM,3,2,9,5
ADEODATUSS,1,2,19,2
BELDIYA00,3,4,18,2
JustAnotherL,0,2,12,1
MasaIDK_,5,1,12,1
OzzieOzz,2,1,15,1
Spiritix,1,8,9,0
fykseN,1,0,15,1


In [62]:
# Fusionner les données de kills avec l'économie totale de l'équipe de l'attaquant
df_merged_team = pd.merge(kills_df, eco_by_team, how='left', left_on=['total_rounds_played', 'attacker_team_name'], right_on=['total_rounds_played', 'team_clan_name'])

# Filtrer les rounds 1 et 13 (pistol rounds) en les incluant dans une nouvelle catégorie
def categorize_kill(row):
    if row['total_rounds_played'] in [1, 13]:
        return 'Pistol round'
    elif row['current_equip_value'] <= 3500:
        return 'Eco kill'
    elif row['current_equip_value'] <= 18000:
        return 'Force buy kill'
    else:
        return 'Full buy kill'

# Appliquer la fonction de catégorisation
df_merged_team['kill_category'] = df_merged_team.apply(categorize_kill, axis=1)

# Utilisation de pivot_table pour compter les kills par catégorie et par joueur
pivot_df_team = df_merged_team.pivot_table(index='attacker_name', columns='kill_category', aggfunc='size', fill_value=0)

# Afficher le résultat
pivot_df_team

kill_category,Eco kill,Force buy kill,Full buy kill,Pistol round
attacker_name,,,,
-silentGG,1,1,15,0
3PARABELLUM,0,3,11,5
ADEODATUSS,0,5,17,2
BELDIYA00,1,0,24,2
JustAnotherL,0,1,13,1
MasaIDK_,0,1,17,1
OzzieOzz,1,3,14,1
Spiritix,0,1,17,0
fykseN,1,3,12,1


#### Scoreboard

In [156]:
overall_stats = ["total_rounds_played","kills_total","assists_total","deaths_total", "mvps", "headshot_kills_total","3k_rounds_total", "4k_rounds_total", "ace_rounds_total" ,"damage_total","utility_damage_total","enemies_flashed_total","alive_time_total"]

stats = demo1.parse_ticks(overall_stats, ticks=[max_tick])
stats['ADR'] = round(stats['damage_total'] / stats["total_rounds_played"],2)
stats['HS %'] = round((stats['headshot_kills_total'] / stats['kills_total'])*100,2)
stats['KPR'] = round((stats['kills_total'] / stats['total_rounds_played']), 2)
stats['K/D'] = round((stats['kills_total'] / stats['deaths_total']),2)
stats = stats.rename(columns={"headshot_kills_total":"HS","kills_total":"Kills","assists_total":"Assists","deaths_total":"Deaths","3k_rounds_total":'3K', "4k_rounds_total":'4K', "ace_rounds_total":'5K',"damage_total":"Damages","utility_damage_total":"Utility Damages","enemies_flashed_total":"Flashed Ennemies"})
stats = stats.drop(columns={"tick"})
nouvel_ordre_colonnes = [
    'name', 'steamid', 'total_rounds_played', 'Kills', 'Deaths', 'Assists', 
    'K/D', 'Damages', 'ADR', 'KPR', 'HS', 'HS %', '5K', '4K', '3K', 'mvps', 'alive_time_total'
]
stats = stats[nouvel_ordre_colonnes]

In [157]:
stats

,name,steamid,total_rounds_played,Kills,Deaths,Assists,K/D,Damages,ADR,KPR,HS,HS %,5K,4K,3K,mvps,alive_time_total
0,-silentGG,76561198111191178,30,17,21,6,0.81,2001,66.70,0.57,8,47.06,0,0,1,3,1333
1,JustAnotherL,76561198264064168,30,15,23,7,0.65,1933,64.43,0.50,10,66.67,0,0,1,3,1307
2,OzzieOzz,76561198066661071,30,19,19,4,1.00,1892,63.07,0.63,9,47.37,0,1,1,4,1250
3,godofbaldz,76561198259827471,30,26,18,4,1.44,2469,82.30,0.87,3,11.54,0,0,3,3,1155
4,ADEODATUSS,76561199198706117,30,24,18,2,1.33,2089,69.63,0.80,15,62.50,0,0,1,3,1155
5,BELDIYA00,76561198376612283,30,27,17,8,1.59,3213,107.10,0.90,15,55.56,0,1,2,4,1222
6,fykseN,76561198014090099,30,17,20,2,0.85,2010,67.00,0.57,7,41.18,0,0,1,1,1207
7,MasaIDK_,76561198358546254,30,19,24,8,0.79,2333,77.77,0.63,9,47.37,0,1,0,3,1395
8,Spiritix,76561198173360875,30,18,19,8,0.95,2000,66.67,0.60,7,38.89,0,0,2,2,1116
9,3PARABELLUM,76561199074956403,30,19,22,7,0.86,2530,84.33,0.63,9,47.37,0,0,3,4,1495


### TABLE FINALE

#### DF par joueurs

In [146]:
joueurs = pd.DataFrame()
joueurs['name']  = team_V3.loc[(team_V3['team_clan_name'] == 'zobrux') & (team_V3['tick'] == first_round_tick), 'name']

In [123]:
Stats_joueurs = joueurs.merge(stats, on='name', how='left')

In [170]:
Stats_joueurs

,name,steamid,total_rounds_played,Kills,Deaths,Assists,K/D,Damages,ADR,KPR,HS,HS %,5K,4K,3K,mvps,alive_time_total
0,-silentGG,76561198111191178,30,17,21,6,0.81,2001,66.70,0.57,8,47.06,0,0,1,3,1333
1,OzzieOzz,76561198066661071,30,19,19,4,1.00,1892,63.07,0.63,9,47.37,0,1,1,4,1250
2,godofbaldz,76561198259827471,30,26,18,4,1.44,2469,82.30,0.87,3,11.54,0,0,3,3,1155
3,BELDIYA00,76561198376612283,30,27,17,8,1.59,3213,107.10,0.90,15,55.56,0,1,2,4,1222
4,Spiritix,76561198173360875,30,18,19,8,0.95,2000,66.67,0.60,7,38.89,0,0,2,2,1116


In [175]:
flash_assist_count

,assister_name,Flash_assist
0,ADEODATUSS,1
1,JustAnotherL,1
2,godofbaldz,1


In [185]:
Utils_joueurs = joueurs.merge(stats_utils, on='name', how='left')
Utils_joueurs = Utils_joueurs.merge(flash_assist_count, on='name', how='left')
Utils_joueurs['Flash_assist'] = Utils_joueurs['Flash_assist'].fillna(0).astype(int)

In [186]:
Utils_joueurs

,name,steamid,Utility Damages,Flashed Ennemies,Flash_assist
0,-silentGG,76561198111191178,175,7,0
1,OzzieOzz,76561198066661071,75,5,0
2,godofbaldz,76561198259827471,79,14,1
3,BELDIYA00,76561198376612283,257,6,0
4,Spiritix,76561198173360875,230,2,0


In [255]:
teamplay_stats = joueurs.merge(opening_stats,on='name',how='left')


In [256]:
teamplay_stats

,name,entry_attempts(T),entry_successes(T),%_entry_success(T),open_attempts(CT),open_successes(CT),%_open_success(CT)
0,-silentGG,3,1,33.0,1,1,100.0
1,OzzieOzz,2,1,50.0,4,0,0.0
2,godofbaldz,6,4,67.0,5,3,60.0
3,BELDIYA00,1,1,100.0,2,2,100.0
4,Spiritix,3,2,67.0,3,1,33.0


In [257]:
teamplay_stats = teamplay_stats.merge(trade_kills_df, on='name',how='left')
teamplay_stats['Trade Kills'] = teamplay_stats['Trade Kills'].fillna(0).astype(int)
teamplay_stats = teamplay_stats.merge(traded_deaths_df, on='name',how='left')
teamplay_stats['Traded Deaths'] = teamplay_stats['Traded Deaths'].fillna(0).astype(int)

teamplay_stats

#### DF par maps